<a href="https://colab.research.google.com/github/uceikow/DataEngineeringGroupAO/blob/master/snorkel_lf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install snorkel

In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:
%cd /content/drive/My\ Drive/

/content/drive/My Drive


In [16]:
# Install spark-related dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
# !tar xf spark-2.4.5-bin-hadoop2.7.tgz

!pip install -q findspark
!pip install pyspark
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/drive/My Drive/spark-2.4.5-bin-hadoop2.7"
!pip install pyspark
!pip install altair

In [0]:
! pip install spacy

In [2]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [17]:
import findspark
findspark.init()

import pyspark
# get a spark context
sc = pyspark.SparkContext.getOrCreate()
print(sc)

# get the context
sqlContext = pyspark.sql.SparkSession.builder.getOrCreate()
print(sqlContext)

<SparkContext master=local[*] appName=pyspark-shell>


In [0]:
# Display full output rather than just the last line of output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
# Load dataset
import pandas as pd

indian = pd.read_csv("/content/data_indian.csv")
indian['label'] = 'indian'
italian = pd.read_csv("/content/data_italian.csv")
italian['label'] = 'italian'
mexican = pd.read_csv("/content/data_mexican.csv")
mexican['label'] = 'mexican'

In [84]:
# Concat them into one dataset
recipe = pd.concat([indian, italian, mexican],ignore_index = True)
recipe

,Title,Description,label
0,Indian Peanut Stew,"This is an easy, authentic dish from South Asi...",indian
1,Roomali Roti,"There is no leavening in this simple, tender I...",indian
2,Spicy Sweet Potato Salad,It's important to use good mayonnaise in this ...,indian
3,Chicken Saag,The classic Indian chicken and spinach dish ge...,indian
4,Paleo Slow Cooker Pork Loin,Boneless pork loin slowly cooks in a curried f...,indian
...,...,...,...
1495,Taco Stew,Ground beef and onions sauteed with a packet o...,mexican
1496,Chicken Tortilla Soup in the Slow Cooker,Everyone loves using their slow cooker to make...,mexican
1497,Bountiful Garden Zucchini Enchiladas,Fresh zucchini and Monterey Jack cheese filled...,mexican
1498,Bean and Honey Burrito Casserole,Here's a great way to feed burritos to a crowd...,mexican


In [20]:
! pip install nltk

In [0]:
import nltk
from nltk import sent_tokenize, word_tokenize

In [22]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
from nltk.corpus import stopwords
import re
import string

In [0]:
# Clean the dataset
# Lowercase
recipe = recipe.apply(lambda row: row.str.lower())

# Remove digits
recipe['Title'] = recipe.apply((lambda row: ''.join([i for i in row['Title'] if not i.isdigit()])),axis = 1)
recipe['Description'] = recipe.apply((lambda row: ''.join([i for i in row['Description'] if not i.isdigit()])),axis = 1)

# Remove punctuations
recipe['Title'] = recipe.apply((lambda row: ''.join([i for i in row['Title'] if i not in string.punctuation])),axis=1)
recipe['Description'] = recipe.apply((lambda row: ''.join([i for i in row['Description'] if i not in string.punctuation])),axis=1)

# Remove Stopwords
stop = stopwords.words('english')
recipe['Title'] = recipe['Title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
recipe['Description'] = recipe['Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [86]:
recipe.head()

,Title,Description,label
0,indian peanut stew,easy authentic dish south asia appeals wide ra...,indian
1,roomali roti,leavening simple tender indian flatbread bread...,indian
2,spicy sweet potato salad,important use good mayonnaise recipe let cooke...,indian
3,chicken saag,classic indian chicken spinach dish gets richn...,indian
4,paleo slow cooker pork loin,boneless pork loin slowly cooks curried fruit ...,indian


Before splitting the dataset and writing labelling function,  we might want to first get an idea of how our targetting labels look like. This gives us some basic information of how to start building the labelling function.

In [87]:
# Patterns from indian food

# Filter out indian food
recipe_ind = recipe[recipe['label'] == 'indian']

# Word frequency in 'Title'
top_N = 15
titles = recipe_ind.Title.str.cat(sep=' ')
words_in_title = nltk.tokenize.word_tokenize(titles)
word_dist_title = nltk.FreqDist(words_in_title)
print('All frequencies:')
print('=' * 60)
t_freq = pd.DataFrame(word_dist_title.most_common(top_N),
                    columns=['Word_title', 'Frequency'])
print(t_freq)
print('=' * 60)

# Word frequency in 'Description'
des = recipe_ind.Description.str.cat(sep=' ')
words_in_des = nltk.tokenize.word_tokenize(des)
word_dist_des = nltk.FreqDist(words_in_des)
print('All frequencies:')
print('=' * 60)
d_freq = pd.DataFrame(word_dist_des.most_common(top_N),
                    columns=['Word_description', 'Frequency'])
print(d_freq)
print('=' * 60)

All frequencies:
   Word_title  Frequency
0     chicken         92
1       curry         82
2      indian         72
3      masala         30
4        rice         28
5       spicy         25
6     curried         21
7      paneer         20
8     chutney         20
9        soup         20
10      salad         19
11       easy         19
12       lamb         17
13     tomato         17
14     potato         16
All frequencies:
   Word_description  Frequency
0            indian        119
1             curry        101
2              dish         83
3           chicken         80
4            recipe         61
5              rice         53
6             spicy         51
7            spices         50
8              made         48
9             sauce         44
10             easy         41
11         simmered         39
12           yogurt         36
13         potatoes         35
14             make         35


From the frequency list we could see that some of the words, such as indian, curry, chicken, spicy, spiced, masala, etc,  are frequently showed up in indian recipe. We can include these words into labelling function.

Inspection is ongoing. Feel free to explore the indian recipe to get useful information to inform the labelling functions.

As being discussed in group meeting, we split the dataset into training, validation, development and test datasets.

If we do multi-labelling, we need to make sure that all datasets above contains same proportion of the 3 recipes. I decided to have 30% labelled data, in which 10% for dev set, 10% for validation set, and the remaining 10% for test set. We left 70% data to training set.

In [0]:
# Split the dataset
# Use ShuffleStratifiedSplit to ensure same proportion of each dataset
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=3, test_size=0.3, random_state=0)

# Get different labelled data
ind = recipe[recipe['label'] == 'indian']
ind.reset_index(drop=True,inplace=True)
ita = recipe[recipe['label'] == 'italian']
ita.reset_index(drop=True,inplace=True)
mex = recipe[recipe['label'] == 'mexican']
mex.reset_index(drop=True,inplace=True)

# Split function (leave 70% for training)
def shuffle_split(df,sss):
  X = df[['Title','Description']]
  y = df['label']
  for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    return X_train, X_test, y_train, y_test

ind_train, ind_test, ind_train, ind_test = shuffle_split(ind,sss)
ita_train, ita_test, ita_train, ita_test = shuffle_split(ita,sss)
mex_train, mex_test, mex_train, mex_test = shuffle_split(mex,sss)

# From randomly sampled test set get dev set and validation set.
ind_val, ind_dev = ind_test[:50], ind_test[50:100]
ita_val, ita_dev = ita_test[:50], ita_test[50:100]
mex_val, mex_dev = mex_test[:50], mex_test[50:100]
ind_test, ita_test, mex_test = ind_test[100:150], ita_test[100:150], mex_test[100:150]